# Classification - Practical Case
## Initialization

In [786]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

## Data Loading

In [787]:
data = pd.read_csv('data.csv')

In [788]:
data

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


# parametric models:
## generetive models


##  Lable Encoding and Test train splitting

In [789]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

### برای مدل های مولد باید جداگانه توزیع هر گروه فیلد هدف محاسبه شود

In [790]:
data_b = pd.read_csv('data-B.csv')
data_m = pd.read_csv('data-M.csv')

X_C1 = data_b.drop(columns=['diagnosis'], inplace=False)
X_C2 = data_m.drop(columns=['diagnosis'], inplace=False)
encoding = LabelEncoder()
y_C1 = encoding.fit_transform(data_b.iloc[:, 0])
y_C2 = encoding.fit_transform(data_m.iloc[:, 0])

In [791]:

X_train_C1, X_test_C1, y_train_C1, y_test_C1 = train_test_split(X_C1, y_C1, test_size=0.3)
X_train_C2, X_test_C2, y_train_C2, y_test_C2 = train_test_split(X_C2, y_C2, test_size=0.3)
X_test = np.vstack((X_test_C1, X_test_C2))

ytest_1 = np.zeros((108, 1))
ytest_2 = np.ones((64, 1))
y_test = np.vstack((ytest_1, ytest_2))

# y_test = np.stack((y_test_C1,y_test_C2),axis=1)
scaler = StandardScaler().fit(X_C1)
scaler1 = StandardScaler().fit(X_C1)
scaler2 = StandardScaler().fit(X_C2)
X_train_C1, X_train_C2, X_test = scaler.fit_transform(X_train_C1), scaler.fit_transform(
    X_train_C2), scaler.fit_transform(X_test)
print(np.shape(y_test))

(172, 1)


### نسبت گروه های فیلد هدف در داده آموزش برای تعیین احتمال دانش پیشین

In [792]:
p = X_C1.shape[0]/X_C2.shape[0]
p

1.6839622641509433

### generative models

In [793]:
def mean(X):
    return sum(X)/float(len(X))
# m = np.mean(X) ----> numpy method

def stdv(X):
    avg = mean(X)
    variance = []
    for i in range(len(avg)):
        variance.append(sum([(x - avg[i])**2 for x in X[:,i]])/float(len(X[:,i]) - 1))  # 1/(N-1))
    return np.sqrt(np.array(variance))
# std = np.std(X) 1/N

def calculate_probability(x, mean, stdev, prior, l): # P(w!x)
    exponent = math.exp(-((x-mean)**2 / (2 * stdev**2 )))
    return l*prior*(1 / (math.sqrt(2 * math.pi) * stdev)) * exponent


In [794]:
m1 = np.mean(X_train_C1,axis=0)
m2 = np.mean(X_train_C2,axis=0)
s1 = np.std(X_train_C1,axis=0)
s2 = np.std(X_train_C2,axis=0)


In [795]:
# s1 = stdv(X_C1)
# s2 = stdv(X_C2)
# p = 0.5
l1 = 1 # for risk analysis
l2 = 1
y_pre = []
p1 = np.empty([X_test.shape[0], X_test.shape[1]])
p2 = np.empty([X_test.shape[0], X_test.shape[1]])
print(X_test.shape)
print(p1.shape)
print("len(m1):",len(m1))
print("len(X_test):",len(X_test))
for i in range(len(m1)):
    for item in range(len(X_test)):
        p1[item,i] = calculate_probability(X_test[item,i],m1[i], s1[i], p,l1)
        p2[item,i] = calculate_probability(X_test[item,i],m2[i], s2[i], 1-p,l2)
print(p1[0,:])
print(np.mean(p1[0,:]))
for item in range(len(X_test)):
        if np.mean(p1[item,:]) > np.mean(p2[item,:]):
            y_pre.append(0)
        else:
            y_pre.append(1)

(172, 30)
(172, 30)
len(m1): 30
len(X_test): 172
[2.86446663e-01 6.61062107e-01 2.73239992e-01 3.88526581e-01
 6.53419358e-02 2.02591085e-01 3.42598952e-01 2.88703222e-01
 5.86078461e-01 6.57424161e-01 6.63391953e-01 1.93573997e-08
 6.43449930e-01 6.29557209e-01 1.91594788e-01 3.95881823e-01
 3.84465112e-01 1.19205807e-01 6.99399328e-02 4.93514347e-01
 2.65429157e-01 5.01126333e-01 2.53638156e-01 4.02753746e-01
 1.19325786e-02 2.44985854e-01 2.93370998e-01 1.28998843e-01
 2.17512463e-01 3.86304775e-01]
0.33496889948148795


In [754]:
confusion_matrix(y_pre, y_test).T

array([[108,   0],
       [ 64,   0]], dtype=int64)

 ### نتیجه گیری : به نظرم میاد مدل مولد مدل خوبی نیست و همه را در گروه 0 پیش بینی کرده
  یا کد من مشکل داره که بعدا باید بررسی شود


## Naive Bayes Classifiers
#### همان مدل مولد است که از کد داخل پایتون استفاده شده و فرض توزیع گوسی است

In [ ]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB

In [672]:
encoding = LabelEncoder()
y = encoding.fit_transform(data.iloc[:,1])
X = data.drop(columns=['id','diagnosis','Unnamed: 32'],inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [0]:
model = GaussianNB()
model.fit(X_train, y_train)

In [674]:
y_pre = model.predict(X_test)
confusion_matrix(y_pre, y_test).T

array([[100,   5],
       [  7,  59]], dtype=int64)

# discriminative models:
## logestic regression:

In [514]:
from sklearn.linear_model import LogisticRegression

In [515]:
X = X.drop(columns=['diagnosis'])
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.3)

In [516]:
scaler = StandardScaler().fit(X_train)
X_train, X_test = scaler.fit_transform(X_train), scaler2.fit_transform(X_test)


In [517]:
model = LogisticRegression()
model.fit(X_train,y_train)

y_pre = model.predict(X_test)

In [518]:
confusion_matrix(y_pre, y_test).T

array([[98,  0],
       [10, 63]], dtype=int64)

## parametric models:
## risk analysis models:

### # در همه مسائل می تواند استفاده شود. اینجا ما در مثال مدل های مولد استفاده میکنیم.
### ریسک یا همان هزینه در احتمال پیشین ضرب می شود
#### جواب مناسبی نگرفتم چون مدل مولد در این مثال جواب مناسب نداد. شاید به خاطر تعداد بالای ویژگی باشه

## non-parametric models:
### Parzen density estimation


In [761]:
data_b = pd.read_csv('data-B.csv')
data_m = pd.read_csv('data-M.csv')
data_b

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,B,13.540,14.36,87.46,566.3,0.09779,0.08129,0.06664,0.047810,0.1885,...,15.110,19.26,99.70,711.2,0.14400,0.17730,0.23900,0.12880,0.2977,0.07259
1,B,13.080,15.71,85.63,520.0,0.10750,0.12700,0.04568,0.031100,0.1967,...,14.500,20.49,96.09,630.5,0.13120,0.27760,0.18900,0.07283,0.3184,0.08183
2,B,9.504,12.44,60.34,273.9,0.10240,0.06492,0.02956,0.020760,0.1815,...,10.230,15.66,65.13,314.9,0.13240,0.11480,0.08867,0.06227,0.2450,0.07773
3,B,13.030,18.42,82.61,523.8,0.08983,0.03766,0.02562,0.029230,0.1467,...,13.300,22.81,84.46,545.9,0.09701,0.04619,0.04833,0.05013,0.1987,0.06169
4,B,8.196,16.84,51.71,201.9,0.08600,0.05943,0.01588,0.005917,0.1769,...,8.964,21.96,57.26,242.2,0.12970,0.13570,0.06880,0.02564,0.3105,0.07409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,B,14.590,22.68,96.39,657.1,0.08473,0.13300,0.10290,0.037360,0.1454,...,15.480,27.27,105.90,733.5,0.10260,0.31710,0.36620,0.11050,0.2258,0.08004
353,B,11.510,23.93,74.52,403.5,0.09261,0.10210,0.11120,0.041050,0.1388,...,12.480,37.16,82.28,474.2,0.12980,0.25170,0.36300,0.09653,0.2112,0.08732
354,B,14.050,27.15,91.38,600.4,0.09929,0.11260,0.04462,0.043040,0.1537,...,15.300,33.17,100.20,706.7,0.12410,0.22640,0.13260,0.10480,0.2250,0.08321
355,B,11.200,29.37,70.67,386.0,0.07449,0.03558,0.00000,0.000000,0.1060,...,11.920,38.30,75.19,439.6,0.09267,0.05494,0.00000,0.00000,0.1566,0.05905


In [763]:
X_C1 = data_b.drop(columns=['diagnosis'], inplace=False)
X_C2 = data_m.drop(columns=['diagnosis'], inplace=False)
encoding = LabelEncoder()
y_C1 = encoding.fit_transform(data_b.iloc[:, 0])
y_C2 = encoding.fit_transform(data_m.iloc[:, 0])
# X_C = np.vstack((X_C1, X_C2))
# y_C = np.vstack((y_C1, y_C2))
X_train_C1, X_test_C1, y_train_C1, y_test_C1 = train_test_split(X_C1, y_C1, test_size=0.3)
X_train_C2, X_test_C2, y_train_C2, y_test_C2 = train_test_split(X_C2, y_C2, test_size=0.3)

In [764]:
X_test = np.vstack((X_test_C1,X_test_C2))


In [765]:
ytest_1 = np.zeros((108, 1))
ytest_2 = np.ones((64, 1))
y_test = np.vstack((ytest_1, ytest_2))
# y_test = np.stack((y_test_C1,y_test_C2),axis=1)

scaler = StandardScaler().fit(X_C1)
scaler1 = StandardScaler().fit(X_C1)
scaler2 = StandardScaler().fit(X_C2)
X_train_C1, X_train_C2, X_test = scaler.fit_transform(X_train_C1), scaler.fit_transform(X_train_C2), scaler.fit_transform(X_test)
print(np.shape(y_test))


(172, 1)


In [766]:
N1 =  X_train_C1.shape[0]
N2 =  X_train_C2.shape[0]
pw1 = N1/(N1+N2)
pw2 = N2/(N1+N2)

In [767]:
class Parzen(object):
    def __int__(self):
        self.x_train = None

    def fit(self,x_train,h):
        self.h = h
        self.x_train = x_train
        self.dimension = self.x_train.shape[1]

    def __kernel(self,x):
        self.__distance = (x.reshape(1,self.dimension) - self.x_train)/self.h

    def __cal_P(self,x):
        self.__kernel(x)
        index = np.where(np.abs(self.__distance < 0.50))
        self.__distance[index[0]] == 1
        self.__distance[self.__distance !=1] = 0
        self.sum = np.sum(self.__distance)
        return self.sum/(len(self.x_train)*(self.h**self.dimension)) # sum(k)/(N*h^D)

    def predict(self,x):
        y_pre = self.__cal_P(x)
        return y_pre



In [768]:
model1 = Parzen()
model2 = Parzen()
h = 0.1
model1.fit(X_train_C1,h)
model2.fit(X_train_C2,h)

In [769]:
y_pre = []
for i in range(len(X_test)):
    p1 = model1.predict(X_test[i, ]) # p(x!w1)
    p2 = model2.predict(X_test[i, ]) # p(x!w2)
    if pw1*p1 >= pw2*p2: # here P(w1) = P(w2) if P(w1) =! P(w2) then if P(w1)*p(x!w1) > P(w2)*p(x!w2)
        y_pre.append(0)
    else:
        y_pre.append(1)

In [770]:
confusion_matrix(y_pre, y_test)


array([[108,  64],
       [  0,   0]], dtype=int64)

 ### از روش پارزن هم جواب نگرفتم. احتمالا این جدا کردن ورودی های باعث مشکل میشه و مشکل از کد منه

## KNN:

In [775]:
from sklearn.neighbors import KNeighborsClassifier

In [776]:
encoding = LabelEncoder()
y = encoding.fit_transform(data.iloc[:, 1])
X = data.drop(columns=['diagnosis','id', 'Unnamed: 32'], inplace=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(np.shape(y_test))

(171,)


In [777]:
model = KNeighborsClassifier(n_neighbors=10)
model.fit(X_train, y_train)
y_pre = model.predict(X_test)

In [778]:
confusion_matrix(y_pre, y_test)

array([[108,   7],
       [  6,  50]], dtype=int64)

## Linear discriminative models:
### perceptron:

In [779]:
from sklearn.linear_model import Perceptron


In [780]:
model = Perceptron()
model.fit(X_train,y_train)

Perceptron()

In [781]:
y_pre = model.predict(X_test)

In [782]:
confusion_matrix(y_pre, y_test)


array([[75,  1],
       [39, 56]], dtype=int64)

### Linear Discriminant Analysis (Fisher)

In [783]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [784]:
model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [785]:
y_pre = model.predict(X_test)
confusion_matrix(y_pre, y_test)


array([[113,   7],
       [  1,  50]], dtype=int64)